# 1. Extracción y transformación de datos

In [147]:
# %pip install tabula-py, para leer tablas de pdfs
# %pip install PyMuPDF, para pdfs
# %pip install xmltodict
# %pip install requests
#%pip install selenium
#%pip install webdriver-manager

In [148]:
import re # utilizar expresiones regulares
import pandas as pd
import requests
import json
import xmltodict #para convertir archivos xml a diccionarios
import time

# import fitz  # Esta es la librería pymupdf que habéis instalado
# from tabula import read_pdf # para abrir pdf y leer tablas con pandas



In [149]:
# base_url = "https://api-tasks.cosasdedevs.com/v1/"
# response = requests.request("POST", url, headers=headers, data=user)
# print_response(response)

### Extraer y transformar datos de un archivo .xml

In [150]:
prescipcionVET = open("../data/PrescipcionVET/PrescripcionVET.xml","r")
xml_prescipcionVET= prescipcionVET.read().replace("&#x2;", ' ')
prescipcionVET_dict =xmltodict.parse(xml_prescipcionVET)
prescipcionVET_dict.keys()

dict_keys(['aemps_prescripcion_vet'])

In [230]:
bloquePrescripcion = prescipcionVET_dict['aemps_prescripcion_vet']['prescription']
gatosCode = 'Gatos'
perrosCode = 'Perros'

prescripciones = []

for prescripcionDict in bloquePrescripcion:
    if not 'dosisrecomendadaespecie' in prescripcionDict:
        continue
    for dosisEspecieDict in prescripcionDict['dosisrecomendadaespecie']:
        if type(dosisEspecieDict) == str:
            continue

        if dosisEspecieDict['categoria'] == gatosCode or dosisEspecieDict['categoria'] == perrosCode:
            prescripciones.append(prescripcionDict)
            break
    
len(prescripciones)

3115

In [152]:
# prescipcionVET_dict['aemps_prescripcion_vet']['prescription'] = prescripcionesGatos

In [231]:
gatosCode = 'Gatos'
perrosCode = 'Perros'
tuResultado = {
    "Medicamento":[],
    "Comercializado":[],
    "Especie de destino":[]
    }
for prescripcionDict in prescripciones:
    if not 'dosisrecomendadaespecie' in prescripcionDict:
        continue
    for dosisEspecieDict in prescripcionDict['dosisrecomendadaespecie']:
        if type(dosisEspecieDict) == str:
            continue
        
        if dosisEspecieDict['categoria'] == gatosCode or dosisEspecieDict['categoria'] == perrosCode:
            tuResultado['Medicamento'].append(prescripcionDict['nombre_med'])
            tuResultado['Comercializado'].append(prescripcionDict['comercializado'])
            tuResultado['Especie de destino'].append(dosisEspecieDict['categoria'])
            break
    

tuResultado

{'Medicamento': ['AEROFAR',
  'CLAVUBAY 500 mg COMPRIMIDOS SABOR PARA PERROS',
  'CLAVUBAY 500 mg COMPRIMIDOS SABOR PARA PERROS',
  'CLAVUBAY 500 mg COMPRIMIDOS SABOR PARA PERROS',
  'CLAVUBAY 500 mg COMPRIMIDOS SABOR PARA PERROS',
  'ADOEQMINA 10 mg/ml SOLUCION CUTANEA # ADOEQMINA',
  'LOXICOM 5 mg/ml SOLUCION INYECTABLE PARA PERROS Y GATOS',
  'LOXICOM 5 mg/ml SOLUCION INYECTABLE PARA PERROS Y GATOS',
  'LOXICOM 5 mg/ml SOLUCION INYECTABLE PARA PERROS Y GATOS',
  'ALSIR 50 mg/ml SOLUCION INYECTABLE # ALSIR 5% SOLUCION INYECTABLE',
  'ALSIR 50 mg/ml SOLUCION INYECTABLE # ALSIR 5% SOLUCION INYECTABLE',
  'ALSIR 50 mg/ml SOLUCION INYECTABLE # ALSIR 5% SOLUCION INYECTABLE',
  'ALUSPRAY',
  'ANTISEDAN',
  'ATOPICA 100 mg CAPSULAS BLANDAS PARA PERROS',
  'ATOPICA 100 mg CAPSULAS BLANDAS PARA PERROS',
  'ATOPICA 100 mg CAPSULAS BLANDAS PARA PERROS',
  'ATOPICA 50 mg CAPSULAS BLANDAS PARA PERROS',
  'ATOPICA 50 mg CAPSULAS BLANDAS PARA PERROS',
  'ATOPICA 50 mg CAPSULAS BLANDAS PARA PERROS',

In [232]:
data_meds = pd.DataFrame(tuResultado)
data_meds

,Medicamento,Comercializado,Especie de destino
0,AEROFAR,SI,Perros
1,CLAVUBAY 500 mg COMPRIMIDOS SABOR PARA PERROS,NO,Perros
2,CLAVUBAY 500 mg COMPRIMIDOS SABOR PARA PERROS,NO,Perros
3,CLAVUBAY 500 mg COMPRIMIDOS SABOR PARA PERROS,NO,Perros
4,CLAVUBAY 500 mg COMPRIMIDOS SABOR PARA PERROS,NO,Perros
...,...,...,...
3110,COXATAB 100 mg COMPRIMIDOS MASTICABLES PARA PE...,NO,Perros
3111,KESIUM 200 mg / 50 mg COMPRIMIDOS MASTICABLES ...,NO,Perros
3112,KESIUM 40 mg / 10 mg COMPRIMIDOS MASTICABLES P...,NO,Gatos
3113,"KESIUM 50 mg / 12,5 mg COMPRIMIDOS MASTICABLES...",NO,Perros


### Extraer y transformar datos de una web

In [171]:
from selenium.webdriver.chrome.options import Options
from webdriver_manager.chrome import ChromeDriverManager
from selenium import webdriver

import warnings
warnings.filterwarnings('ignore')

from selenium.webdriver.common.by import By

In [180]:
def crearDriver():
    opciones=Options()

    opciones.add_experimental_option('excludeSwitches', ['enable-automation'])
    opciones.add_experimental_option('useAutomationExtension', False)
    opciones.headless=False    # si True, no aperece la ventana (headless=no visible)
    opciones.add_argument('--start-maximized')         # comienza maximizado
    #opciones.add_argument('user-data-dir=selenium')    # mantiene las cookies
    #opciones.add_extension('driver_folder/adblock.crx')       # adblocker
    opciones.add_argument('--incognito')
    # opciones.add_argument('user-data-dir=cookies')    # mantiene las coockies

    driver = webdriver.Chrome(options = opciones)
    return driver



In [197]:
def extraerRazas(driver, url: str)->list:
    """Example function with PEP 484 type annotations.

    Args:
        driver: .
        url: .

    Returns:
            razas

    """
    driver.get(url)
    time.sleep(3)


    cookiesButton = driver.find_element(By.XPATH, '//*[@id="onetrust-accept-btn-handler"]')
    if cookiesButton:
        cookiesButton.click()
        time.sleep(2)

    razas = []

    while True:
        gatos = driver.find_elements(By.CLASS_NAME, 'results-view-name')
        if not gatos:
            break
        razas = razas + [e.text for e in gatos] #extend

        next_page_button = [x for x in driver.find_element(By.CLASS_NAME, 'pager').find_elements(By.CSS_SELECTOR, 'a') if x.get_attribute('title')=='Go to next page']

        if not next_page_button:   # Si no lo encuentra, no hay más páginas
            break
        try:
            next_page_button[0].click()
        except:
            next_page_button[0].click()
        finally:
            time.sleep(3)

    driver.quit()
    return razas


In [192]:
driver = crearDriver()
razasGato = extraerRazas(driver, url='https://www.purina.es/encuentra-mascota/razas-de-gato?page=%2C0')

In [193]:
print(razasGato)

['Abisinio', 'Americano de pelo duro', 'Asiático', 'Azul ruso', 'Balinés', 'Bengalí', 'Birmano', 'Bobtail japonés de pelo corto', 'Bobtail japonés de pelo largo', 'Bombay', 'Bosque de Noruega', 'Bosque de Siberia', 'Británico de pelo corto', 'Burmés', 'Burmilla', 'Chinchilla', 'Cornish rex', 'Cymric', 'Devon Rex', 'Exótico de pelo corto', 'Fold escocés', 'Khao Manee', 'Korat', 'Laperm', 'Maine coon', 'Manx', 'Mau egipcio', 'Mist australiano', 'Munchkin', 'Ocigato', 'Oriental de pelo corto', 'Oriental de pelo largo', 'Persa de pelo largo', 'Pixie bob', 'Ragdoll', 'Savannah', 'Selkirk rex', 'Siamés', 'Singapura', 'Snowshoe', 'Somalí', 'Sphynx', 'Tiffanie', 'Tonquinés', 'Van turco']


In [216]:
gatos = pd.DataFrame(razasGato, columns=['Razas de Gatos'])
gatos.head(10)

,Razas de Gatos
0,Abisinio
1,Americano de pelo duro
2,Asiático
3,Azul ruso
4,Balinés
5,Bengalí
6,Birmano
7,Bobtail japonés de pelo corto
8,Bobtail japonés de pelo largo
9,Bombay


In [198]:
driver = crearDriver()
razasPerro = extraerRazas(driver, url='https://www.purina.es/encuentra-mascota/razas-de-perro')

In [199]:
print(razasPerro)

['Affenpinscher', 'Afgano', 'Akita japonés', 'Basenji', 'Basset Azul de Gascuña', 'Basset Grifón vandeano (grande)', 'Basset Grifón vandeano (pequeño)', 'Basset Hound', 'Basset leonado de Bretaña', 'Beagle', 'Beauceron', 'Bedlington Terrier', 'Bergamasco', 'Bichón Boloñés', 'Bichón Frisé', 'Bloodhound', 'Bobtail (antiguo perro pastor inglés)', 'Border Collie', 'Border Terrier', 'Borzoi', 'Boston Terrier', 'Bóxer', 'Boyero de Berna', 'Boyero de Flandes', 'Bracco italiano', 'Braco alemán de pelo corto', 'Braco alemán de pelo duro', 'Braco de Weimar (de pelo corto y suave)', 'Braco húngaro', 'Braco húngaro de pelo duro', 'Bretón', 'Buhund noruego', 'Bull Terrier', 'Bull Terrier miniatura', 'Bulldog', 'Bulldog francés', 'Bullmastiff', 'Cairn Terrier', 'Caniche enano', 'Caniche grande', 'Caniche toy', 'Carlino', 'Cavalier King Charles Spaniel', 'Cazador de alces noruego (elkhound noruego)', 'Chihuahua (de pelo largo)', 'Chihuahua (de pelo suave)', 'Chin japonés', 'Chow chow (de pelo duro)',

In [214]:
perros = pd.DataFrame(razasPerro, columns=['Razas de Perros'])
perros

,Razas de Perros
0,Affenpinscher
1,Afgano
2,Akita japonés
3,Basenji
4,Basset Azul de Gascuña
...,...
184,Vallhund sueco
185,West Highland White Terrier
186,Whippet
187,Xoloitzcuintle (mediano)


### Extraer y transformar datos de un archivo .csv

In [235]:
data_censos = pd.read_csv('../data/censo_animales.csv', sep=';', names=['AÑO', 'DISTRITO', 'ESPECIE CANINA', 'ESPECIE FELINA'])

In [236]:
data_censos.head()

,AÑO,DISTRITO,ESPECIE CANINA,ESPECIE FELINA
0,AÑO,DISTRITO,ESPECIE CANINA,ESPECIE FELINA
1,2021,ARGANZUELA,10843,5439
2,2021,BARAJAS,5139,1623
3,2021,CARABANCHEL,20595,6926
4,2021,CENTRO,16442,9878


Elimino la primera fila y reestablezco el índice

In [238]:
data_censos = data_censos.iloc[1:]
data_censos = data_censos.reset_index(drop=True)
data_censos


,AÑO,DISTRITO,ESPECIE CANINA,ESPECIE FELINA
0,2021,ARGANZUELA,10843,5439
1,2021,BARAJAS,5139,1623
2,2021,CARABANCHEL,20595,6926
3,2021,CENTRO,16442,9878
4,2021,CHAMARTÍN,11361,4122
...,...,...,...,...
163,2014,TETUÁN,12301,2178
164,2014,USERA,11310,978
165,2014,VICÁLVARO,4584,505
166,2014,VILLA DE VALLECAS,7107,940


## Resumen

- Extraer y transformar datos de un archivo .xml

- Extraer y transformar datos de una web

- Extraer y transformar datos de un archivo .csv